In [1]:
from __future__ import print_function
import os
import sys
import requests
from operator import add

from pyspark import SparkConf,SparkContext
from pyspark.streaming import StreamingContext

from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

from pyspark.sql.types import *
from pyspark.sql import functions as func
from pyspark.sql.functions import *
from pyspark.ml.linalg import Vectors

if __name__ == "__main__":
    if len(sys.argv) != 4:
        print("Usage: main_task1 <file> <output> ", file=sys.stderr)
        exit(-1)
    sc = SparkContext()
    spark = SparkSession.builder.getOrCreate()
    rdd = sc.textFile(sys.argv[1])
    file="file:/Applications/BU MET Course Material/CS777/Assignment/Ma_Mingjun_Project/2020_02.csv"
    data = SQLContext(sc).read.format('csv').options(header='true', inferSchema='true',  sep =",").load(file)
    data_in_rdd = data.rdd.map(tuple)

Usage: main_task1 <file> <output> 
22/08/16 16:32:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
    delay_time = data_in_rdd.map(lambda x: (x[1], x[3], x[5], x[9], x[8]))
    # delay time <- train id, previous station, next station, delay, time 
    delay_time = delay_time.filter(lambda x: x[1]!=x[2])
    delay_time = delay_time.filter(lambda x: x[3]!=None)
    northeast = ['3806','3808','3700','3910','3810','3702','3812','3704','3914','3814','3706','3918','3818','3708',
                        '3920','3124','3922','3710','3924','3126','3712','3926','3828','3714','3930','3830','3716',
                        '3932','3718','3720','3934','3936','3722','3834','8838','8840','3836','3838','8842',
                        '3840','3842','3844','3846','3848','3850','3852','3154','3954','3856','3858','3860',
                        '3896','3736','3864','3738','3866','3898','3868','3870','3872','3874','3876',
                        '3880','3984','3882','3886','3742','3888','3892','3800','3809','3813',
                        '3701','3815','3705','3817','3709','3821','3823','3713','3825','3827','3915','3917','3829',
                        '3831','3833','3835','3837','3839','3841','3843','3845','3847','8849','3849','8851', 
                        '3719','3937','3753','8855','3855','3857','3721','3943','3723','3861','3947','3165','3949',
                        '3725','3951','3867','3727','3953','3871','3957','3959','3873','3175','3961','3875','3963',
                        '3733','3281','3881','3965','3883','3969','3885','3285','3887','3889','3289','3737','3891','3293','3893','3895', 
                        '3297','3201','3897','7801','3205','3805','7204','7804','8800','7808','7208','8804','8810','8312','7812','7814',
                        '7816','8816','7818','7620','7820','7822','7624','7824','7826','7828','7830','7832',
                        '7834','7836','7838','7840','7844','7846','7848','7850','7852','7854',
                        '7856','7858','7860','7864','7868','7872','7876','7880',"3955",'7800',
                        '7813','7817','7821','7625','7225','7825','7629','7229','7829','7233','7833','7637','7237','7837',
                        '7839','7241','7841','8841','7245','7845','7847','7249','7849','7851','7253','7853','7855','7657','7257','7857','8325',
                        '7859','7261','7861','8861','7863','7665','7265','7865','8865','7669','7867','7269','7869','7673','7871','7273','7873', 
                        '7875','7277','7877','7879','7281','7881','7883','7285','7885','7201','7887','7801','7205','7805','8700']
    atl = list(set(['4608','4610','4612','4616','4620','4624','4628','4630','4632','4634','4638','4642','4613','4615','4617','4623','4627',
                        '4631','4633','4635','4639','4641','4643','4651','4660','4662','4664','4668','4670','4674','4678','4680','4682','4684',
                        '4688','4692','4661','4669','4671','4673','4677','4679','4683','4685','4687','4689','4693','4697'
                    ]))
    main_bergen = list(set(['1100','42','1102','1146','44','1148','1202','1150','46','1204','1152','48','50','1252','1206','1154','52','1254','1208',
                        '1156','1158','54','1210','1160','56','1212','1162','1110','1264','58','1112','1266','1114','1168','1116','62','1118','1272',
                        '1120','64','1174','1220','1122','66','1178','1124','1274','1126','1128','1276','1130','1278','1132','1182','1134','1184',
                        '1136','1186','68','1234','1103','1245','1105','1247','1107','1147','43','1249','1109','1207','45','1351','1111','1353','1113', 
                        '1155','47','1211','1157','1213','1159','49','99','1115','1161','1215','51','1263','1117','1319','53','1165','1357','1221','55',
                        '1167','1269','57','1223','1169','1271','59','1225','1171','61','1173','1125','63','1175','1127','65','1177','1129','1179','67',
                        '1281','1131','1183','1133','41','1101',
                        '1790','1792','70','1752','1794','1704','1854',
                        '94','1706','1856','72','1708','1858','1710','74','1712','1862','1714','76','1716','1866','1718','1868','1720','1870','1722','1872',
                        '78','1724','1874','1726','1876','80','1728','1878','1730','1880','1732','1882','82','1784','1786','1855','1705','1857','1707','1859',
                        '1709','71','1711','1863','1713','73','1715','1867','1717','1869','75','1719','1871','1721','1873','1723','77','1875','1725','1793',
                        '1877','93','1727','1795','1797','79','1879','1729','1881','1731','1883','1733','1835','81','1837','1787','1839','1789','1801','69',
                        '1701']))
    #"Main/Bergen County"
    #"Montclair- Boonton"
    montclair_boonton = list(set(["6200","6202","6204","1070","204","6206","1000","852","208","6210","1002","872","212","6214","1006","808","6216","1074","6222",
                        "6226","6230","6234","854","6238","6242","246","6248","6252","880","6258","262","858","6264","6270","882","6274","884","6278",
                        "6280","871",  "6209","211","6213","6215","6219","6227","851","6231","6233","6237","6241","6245","1001","877","6251","1003",
                        "6253","853","1079","6263","1055","267","807","6273","1009","275","881","6279","1011","6283","1085","6291","6293","1087","6295",
                        "1061","6297","6299","6201","1090", "510","1092","522","530", "534","538","544","8404","9200","6910","8908",
                        "9202","1094","6914","6918","6922","6926","6930","6934","6938","6944","6915","6919","6923","6927","6931","8431","6935","6939", 
                        "6945","6909","519","523","527","1091","531","9227","1093","9229","1095","535","539","545","509","1090", "9290", 
                        "1092","9292","9204","514","9208","518","9212","9216","526","9228","9230","9530","9232","9236","9540","9544","9225",
                        "8404","6910","8908","6918","6926","6934","6940","6944","6923","6927","6931","8431","965","6935", "6939","6945","6909",
                        "513","9213","515","9215","9219","9221","9223","9225","531","9297","1093","9299","1095","535","9237","9539","9241","9545","9509"]))
    # "Morris & Essex":
    morris_essex = list(set(["6200","6602","300","400","6202","6604","402","6204","6606","1070","608","404","6610","204","6406","6612","6206","308","408","1000","852",
                        "6314","6614","208","6410","312","412","1002","6616","6318","872","212","414","6620","6214","1006","6320","808","416","6624","6324","6216",
                        "626","480","6628","1074","6328","6222","6332","6632","418","6226","6636","420","6230","6640","422","6234","854","6644","424","6238","6648",
                        "426","6242","6650","428","6654","246","6656","6248","430","6658","660","6252","432","6662","330","6664","880","6258","6666","262","334",
                        "858","6668","6264","336","6670","6270","882","6672","434","674","6274","6674","884","438","6676","6278","682","6280","440","442","301","871",
                        "6209","6305","607","211","6307","6213","481","6607","309","6215","6611","409","6613","6219","6315","6617","411","6227","851","6317","413",
                        "6231","6619","6233","415","6621","6237","417","6625","6241","419","6627","6245","421","6631","1001","8935","877","6635","423","6251","6339",
                        "6639","485","1003","6253","853","6341","8941","6641","427","1079","643","6343","6643","6263","645","429","1055","6647","6431","267","807","6273",
                        "433","6651","337","1009","6653","275","881","435","339","6279","6355","6655","1011","657","6283","439","6359","6659","6363","1085","6663",
                        "665","6291","443","6667","6293","447","6671","1087","6295","449","6675","1061","6677","6297","0451","6679","6299","453","6683","6201","6601",
                        "8906","6908","952","1090","8404","710","6910","8406","1092","8908","712","6912","954","956","714","716","6916","718","6918",
                        "518","720","6920","722","724","726","6924","6926","526","728","730","6928","6930","530","962","732","734","736","738","740","742","744", 
                        "6932","964","6934","534","966","6936","6938","538","6940","6942","6944","544","746","951","8911","711","953","955","6913","6915","6917","6919",
                        "6921","6923","6925","6927","6929","961","713","715","719","721","723","725","727","729","531","6931","963","8431","1093","6933","8933","965","1095","8433",
                        "535","6935","6937","539","6939","6941","6943","545","6945","6907","509","6909","731","733","735","737","739","741","743","745","707","709","8404","6910","8406","1092",
                        "8908","9208","6918","518","9212","736","6437"
                    ]))
    #   "North Jersey Coast Line"
    NorthJersey = list(set(["3202","3204","3208","3210","3312","3218","3502","3320","3222","3504","3224","3506","3326","3228","3230","3508","3234","8234","3236","3240",
                                            "4310","4324","4330","4334","4340","4344","4352","4360","4366","4372","4378","4384","4392","4398","3244","3248","3252","3256","3260","3510", 
                                            "3266","3598","3270","3512","3272","3514","3276","3516","3278","3284","3288","3292","3200","3209","3215","3503","3217","3505","3221","3223",
                                            "3227","3231","3235","3243","8245","3247","8249","3251","8353","3509","3255","3595","4315","4321","4323","4331","4335","4343","4351",
                                            "4355","3361","3363","3511","3265","3513","3267","3271","3373","3515","3275","3279","3283","3285","3289","3293","3297","3201","3205",
                                            "4367","4375","4379","4383","4393","4397","4708","4716","4720","4724","4728","4732","4736","4740","4744","4748","4752","4756","4760","4764",
                                            "4768","4772","4780","7204","7208","8312","7212","7216","7220","7224","7228","7232","7632","7236","7240","7244","7248","7652","7252","7256",
                                            "7660","7260","7264","7668","7268","7272","7276","7200","7217","3239"]))
    #   "Pascack Valley"
    pascackValley = list(set(["1602","1604","1606","1608","1610","1612","1614","1616","1618","1620","1650","1622","1652","1624","1626","1628","1630","1632","1634","1636",
    "1605","1651","1607","1611","1613",":1617","1621","1653","1623","1625","1627","1629","1631","1633","1635","1637","1641","1643","1645","1647","2101",
    "2100","2102","2104","2106","2108","2110","2112","2114","2118","2122","2124","2126","2103","2107","2111","2115","2117","2119","2121","2123","2125","2129","2133","2101",
    "9690","9600","9602","9692","9604","9606","9608","9612","9614","9618","9626","9603","9607","9611","9615","9619","9621","9693","9623","9625","9629","9633"]))
    # Raritan Valley
    raritan_valley = list(set(["5404","2406","5408","5710","5412","5714","5416","5718","5420","5422","5902","5126","5170","5132","5134","5176","5438","5440","5742", 
    "5444","5746","5448","5150","5152","5154","5456","5458","5409","5711","5413","5415","5719","5121","5123","5175","5127","5179","8529","5431","5733","5435",
    "5737","5439","5441","5743","5445","8545","5747","5451","5193","5155","5197","5401","5403","5506","8512","5508","5510","5512","5514","5516","5518","5520","5522","5524","5526",
    "5528","5530","5532","5534","5536","5538","5540","5511","5513","5515","5517","5519","8521","5521","5523","5525","5527","5529","8551","5531","5533","5535","5537","5539","5541","5543","5503"]))
                        

In [3]:
    delay_time = delay_time.map(lambda x: (x[0], x[1], x[2], x[3], x[4][0:11], int(x[4][11:13]), int(x[4][14:16]),int(x[4][17:]))).filter(lambda x: x[3]!=0)

In [4]:
    # I believe delaying over 5 minutes is intense    
    top_delay = delay_time.filter(lambda x: x[3]>=5)
    print(top_delay.count())

48565


In [5]:
    top_delay_list1 = delay_time.top(top_delay.count(), lambda x:(x[3]))

In [6]:
    nec_rdd = sc.parallelize(northeast).map(lambda x: (x,"Northeast Corridor"))
    atl_rdd = sc.parallelize(atl).map(lambda x: (x,"Atlantic City"))
    main_bergen_rdd = sc.parallelize(main_bergen).map(lambda x: (x,"Main/Bergen County"))
    mont_clair_boson_rdd = sc.parallelize(montclair_boonton).map(lambda x:(x,"Montclair- Boonton"))
    northjersey_rdd = sc.parallelize(NorthJersey).map(lambda x:(x,"North Jersey Coast"))
    morris_essex_rdd = sc.parallelize(morris_essex).map(lambda x:(x,"Morris Essex"))
    pascackValley_rdd = sc.parallelize(pascackValley).map(lambda x:(x,"Pascack Valley"))
    r_valley_rdd = sc.parallelize(raritan_valley).map(lambda x:(x,"Raritan Valley"))

In [7]:
    schedule = nec_rdd.union(northjersey_rdd)
    schedule = schedule.union(r_valley_rdd)
    schedule_north= main_bergen_rdd.union(morris_essex_rdd)
    schedule_north = schedule_north.union(pascackValley_rdd)
    schedule = schedule.union(schedule_north)
    schedule = schedule.union(mont_clair_boson_rdd)
    schedule = schedule.union(atl_rdd)
    schedule_dict = dict(schedule.collect())

In [8]:
    top_delay_rdd = sc.parallelize(top_delay_list1)

In [9]:
    schedule_join = top_delay_rdd.join(schedule).map(lambda x:(x[0],x[1][1]))
    schedule_df = spark.createDataFrame(schedule_join,schema = ["Train ID","Line"])

In [10]:
    print(schedule_df.groupBy("Line").count().show())

+------------------+-----+
|              Line|count|
+------------------+-----+
|Montclair- Boonton| 4347|
|     Atlantic City| 2243|
|    Raritan Valley| 3825|
|North Jersey Coast| 5840|
|Northeast Corridor| 9479|
|Main/Bergen County| 3858|
|      Morris Essex|10169|
|    Pascack Valley| 2517|
+------------------+-----+

None


In [11]:
    top_delay_rdd = sc.parallelize(top_delay_list1)
    delay_time_rdd = spark.createDataFrame(top_delay_rdd, schema = ["Train ID","Start","End","Delay Minutes","Date","Hour","Min","Sec"])

In [12]:
    r4 = delay_time_rdd.groupBy("Hour").count()
    r4 = r4.rdd.map(tuple)
    print(r4.top(3, key =lambda x: x[1]))

[(18, 5158), (19, 4599), (17, 3391)]


In [13]:
    #the worst delayed trains 
    delay_max = dict()
    for i in range(50):
        if top_delay_list1[i][0] in schedule_dict:
            print("Train", top_delay_list1[i][0], "from", schedule_dict[top_delay_list1[i][0]], "delays", top_delay_list1[i][3],"minutes")
            if schedule_dict[top_delay_list1[i][0]] not in delay_max:
                delay_max[schedule_dict[top_delay_list1[i][0]]] = 0
            delay_max[schedule_dict[top_delay_list1[i][0]]] +=1 

Train 9225 from Montclair- Boonton delays 149.28333333333333 minutes
Train 9225 from Montclair- Boonton delays 148.55 minutes
Train 9225 from Montclair- Boonton delays 148.48333333333332 minutes
Train 9225 from Montclair- Boonton delays 148.46666666666667 minutes
Train 9225 from Montclair- Boonton delays 148.41666666666666 minutes
Train 9225 from Montclair- Boonton delays 148.03333333333333 minutes
Train 9225 from Montclair- Boonton delays 147.76666666666668 minutes
Train 9225 from Montclair- Boonton delays 147.75 minutes
Train 9225 from Montclair- Boonton delays 147.58333333333334 minutes
Train 9225 from Montclair- Boonton delays 147.3 minutes
Train 9225 from Montclair- Boonton delays 146.0 minutes
Train 9227 from Montclair- Boonton delays 120.93333333333334 minutes
Train 3275 from North Jersey Coast delays 120.0 minutes
Train 3275 from North Jersey Coast delays 120.0 minutes
Train 3275 from North Jersey Coast delays 119.0 minutes
Train 3275 from North Jersey Coast delays 119.0 minute

In [14]:
    print(delay_max)

{'Montclair- Boonton': 21, 'North Jersey Coast': 20, 'Northeast Corridor': 9}


In [4]:
    sc.stop()